In [ ]:
# This architecture mimics the real-world hiring funnel to give candidates actionable feedback on *exactly* where they are getting dropped.

# Here is the robust setup for your **ATS  Recruiter  Hiring Manager** pipeline using **LangGraph** (workflow), **Gemini** (intelligence), and **Pydantic** (structure).

# ---

# ### Phase 1: The Robot (ATS Agent)

# **Goal:** Strict keyword matching, formatting checks, and hard constraints (e.g., years of experience). This agent is "dumb" but precise.

# #### 1. Pydantic Model

# ```python
# from pydantic import BaseModel, Field
# from typing import List

# class ATSResult(BaseModel):
#     match_percentage: int = Field(..., description="0-100 score based on keyword overlap.")
#     missing_keywords: List[str] = Field(..., description="Critical technical keywords found in JD but missing in Resume.")
#     formatting_issues: List[str] = Field(..., description="Issues like complex columns, missing headers, or unreadable fonts.")
#     years_of_experience_detected: float = Field(..., description="Total years of professional experience extracted.")
#     is_hard_filter_passed: bool = Field(..., description="True if mandatory requirements (e.g. '3+ years Python') are met.")

# ```

# #### 2. Robust Gemini Prompt

# ```text
# ROLE: You are an Applicant Tracking System (ATS) Parser.
# GOAL: Parse the candidate's resume against the Job Description (JD) strictly and mechanically. Do not "hallucinate" skills that are implied; if the word isn't there, it doesn't count.

# INSTRUCTIONS:
# 1. Extract Skills: Scan the resume for exact keyword matches from the JD.
# 2. Check Filters: Verify hard requirements (e.g., "Must have Bachelor's", "Must have 5 years exp").
# 3. Audit Formatting: Detect if the resume uses standard headers (Experience, Education). Flag complex tables or graphics that make parsing hard.
# 4. Calculate Score: Base the match percentage purely on keyword density and hard requirements.

# INPUT:
# Resume Text: {resume_text}
# Job Description: {job_description}

# OUTPUT: Return the result ONLY in the defined JSON structure.

# ```

# ---

# ### Phase 2: The Recruiter (Sanity Check Agent)

# **Goal:** Evaluates the "human" elements: career stability, red flags, clarity, and basic eligibility. This agent is non-technical but understands "red flags."

# #### 1. Pydantic Model

# ```python
# class RecruiterFeedback(BaseModel):
#     pass_screening: bool = Field(..., description="Should this candidate move to the Hiring Manager?")
#     red_flags: List[str] = Field(..., description="e.g., Employment gaps > 6 months, job hopping (<1 year tenure multiple times).")
#     soft_skills_rating: int = Field(..., description="1-10 score on communication clarity and leadership potential.")
#     career_trajectory: str = Field(..., description="One sentence summary: e.g., 'Consistent growth from Junior to Senior roles.'")
#     summary_for_hiring_manager: str = Field(..., description="A 2-sentence pitch to the HM about why they should view this.")

# ```

# #### 2. Robust Gemini Prompt

# ```text
# ROLE: You are a Senior Technical Recruiter.
# GOAL: You are the "human filter" between the ATS and the Engineering Manager. You don't know code, but you know what a good employee looks like.

# INSTRUCTIONS:
# 1. Scan for Stability: Look for "Job Hopping" (leaving jobs in <12 months repeatedly) or unexplained gaps.
# 2. Check Progression: Do their titles improve over time? (e.g., Junior -> Senior -> Lead).
# 3. Verify Prestige/Context: Did they work at known companies or startups? (Give bonus points for recognized tech names).
# 4. Evaluate Brevity: Is the resume easy to read, or is it a wall of text?

# CONTEXT:
# ATS Score: {ats_score} (If < 70, be very skeptical unless the resume is exceptional in other ways).

# OUTPUT: Return the result ONLY in the defined JSON structure.

# ```

# ---

# ### Phase 3: The Hiring Manager (Technical Deep Dive Agent)

# **Goal:** Verifies *depth* of knowledge, impact (numbers/results), and specific tech stack usage. This agent is skeptical and technical.

# #### 1. Pydantic Model

# ```python
# class HiringManagerAssessment(BaseModel):
#     technical_depth_score: int = Field(..., description="1-10. Does the candidate understand the 'why' and 'how', or just list tools?")
#     impact_analysis: str = Field(..., description="Analysis of their bullet points. Did they list duties (bad) or achievements (good)?")
#     culture_fit_clues: str = Field(..., description="Evidence of teamwork, mentoring, or open-source contribution.")
#     interview_decision: str = Field(..., description="YES / NO / MAYBE")
#     suggested_interview_questions: List[str] = Field(..., description="3 specific technical questions to drill into their weak spots.")

# ```

# #### 2. Robust Gemini Prompt

# ```text
# ROLE: You are a Grumpy Engineering Manager / CTO.
# GOAL: Evaluate if this person can actually write code and deliver value. You hate buzzwords. You love metrics.

# INSTRUCTIONS:
# 1. Scrutinize "Impact": Ignore lines like "Worked on Java." Look for "Refactored API to reduce latency by 30%." If there are no numbers, penalize heavily.
# 2. Tech Stack Context: Did they use the required tech in a complex environment? (e.g., "Used Redis for caching" vs "Installed Redis").
# 3. Bullshit Detection: Flag if they list 20 languages but have only 2 years of experience.
# 4. Final Decision: Would you spend 1 hour of your expensive time interviewing them?

# INPUT:
# Resume: {resume_text}
# JD: {job_description}

# OUTPUT: Return the result ONLY in the defined JSON structure.

# ```

# ---

# ### The Workflow (LangGraph Implementation)

# Here is how you stitch it all together using LangGraph. This workflow passes the state between the agents.

# ```python
# from typing import TypedDict, Annotated
# from langgraph.graph import StateGraph, END
# from langchain_google_genai import ChatGoogleGenerativeAI

# # 1. Define State
# class AgentState(TypedDict):
#     resume_text: str
#     job_description: str
#     ats_result: ATSResult
#     recruiter_result: RecruiterFeedback
#     hm_result: HiringManagerAssessment

# # 2. Define Nodes (The Agents)
# def ats_agent(state: AgentState):
#     llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
#     structured_llm = llm.with_structured_output(ATSResult)
#     # ... insert prompt from Phase 1 ...
#     response = structured_llm.invoke(formatted_prompt) 
#     return {"ats_result": response}

# def recruiter_agent(state: AgentState):
#     # Only run if ATS score is decent (Optimization)
#     if state["ats_result"].match_percentage < 50:
#         return {"recruiter_result": RecruiterFeedback(pass_screening=False, ...)}
    
#     llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.3)
#     structured_llm = llm.with_structured_output(RecruiterFeedback)
#     # ... insert prompt from Phase 2 ...
#     response = structured_llm.invoke(formatted_prompt)
#     return {"recruiter_result": response}

# def hm_agent(state: AgentState):
#     if not state["recruiter_result"].pass_screening:
#         return {"hm_result": None}

#     llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.5)
#     structured_llm = llm.with_structured_output(HiringManagerAssessment)
#     # ... insert prompt from Phase 3 ...
#     response = structured_llm.invoke(formatted_prompt)
#     return {"hm_result": response}

# # 3. Build Graph
# workflow = StateGraph(AgentState)
# workflow.add_node("ats", ats_agent)
# workflow.add_node("recruiter", recruiter_agent)
# workflow.add_node("hiring_manager", hm_agent)

# workflow.set_entry_point("ats")
# workflow.add_edge("ats", "recruiter")
# workflow.add_edge("recruiter", "hiring_manager")
# workflow.add_edge("hiring_manager", END)

# app = workflow.compile()

# ```

# ### Next Step

# To make this truly "Agentic," do you want me to add a **"Resume Optimizer" node** at the end? If the candidate fails any stage, this agent could automatically rewrite their bullet points to fix the specific error found by the ATS or Hiring Manager.